This is playing with pytorch framework for EHR modeling. In general, a patient's health record can be represented as a sequence of visits. Each visit has certain features, and can be represented as a list of medical codes.

For simplicity, we are starting with the data structure that a patient's health record is a list of list, following the line of work from Jimeng Sun's lab. We will use codes from Ed Choi to manipulate the data. 

The core model is an GRU.

# todos:
* regularization
* save/load model
* load pre-trained embedding
* mini-batching # Done
* make sure it runs on GPU # Done

In [ ]:
%matplotlib inline
from __future__ import print_function, division
from io import open
import string
import re
import random
import sklearn 
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.utils.data as Data

use_cuda = torch.cuda.is_available()

import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

from torchviz import make_dot, make_dot_from_trace

# for windows only    
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'


In [ ]:
use_cuda 

In [ ]:
# prepare data: load the input file containing list of list of list, and corresponding label file
# and output the splitted training, valid and Test sets

def data_load_split_VT(seqFile = 'pytorch_ehr-master/pytorch_ehr-master/data/cerner/hospital_data/h143.visits', labelFile = 'pytorch_ehr-master/pytorch_ehr-master/data/cerner/hospital_data/h143.labels' , test_r=0.2 , valid_r=0.1):

    set_x = pickle.load(open(seqFile, 'rb'), encoding='bytes')
    set_y = pickle.load(open(labelFile, 'rb'),encoding='bytes')
    merged_set = [[set_y[i],set_x[i]] for i in range(len(set_x))] # merge the two lists

    # set random seed
    random.seed( 3 )
    
    dataSize = len(merged_set)
    nTest = int(test_r * dataSize)
    nValid = int(valid_r * dataSize)
    
    random.shuffle(merged_set)

    test_set = merged_set[:nTest]
    valid_set = merged_set[nTest:nTest+nValid]
    train_set = merged_set[nTest+nValid:]

    return train_set, valid_set, test_set

In [ ]:
train_sl , valid_sl , test_sl = data_load_split_VT()

#print (train_sl[0:5])


In [ ]:
## x=[[0, [[1667, 144, 62, 85], [1667, 144, 62, 85]]], [0, [[491, 129, 813, 149, 84, 85], [491, 129, 813, 61, 84, 85], [358, 61, 84, 85], [491, 1250, 813, 61, 84, 85], [499, 61, 84, 85], [499, 61, 84, 85], [8004, 61, 84, 85], [2914, 61, 84, 85], [2914, 61, 84, 85], [2499, 499, 61, 84, 85], [499, 61, 84, 85], [1250, 129, 813, 61, 84, 85], [1250, 61, 84, 85]]], [0, [[2374, 345, 84, 85]]], [0, [[1205, 267, 62, 85]]], [0, [[277, 144, 84, 85], [650, 692, 252, 124, 68, 129, 130, 873, 132, 16, 186, 1016, 187, 1259, 550, 96, 26, 1085, 30, 34, 226, 192, 209, 851, 1121, 177, 144, 84, 85], [2171, 144, 84, 85], [4270, 119, 350, 2171, 458, 843, 129, 130, 1664, 12, 195, 156, 1016, 74, 1259, 550, 7, 78, 1085, 31, 34, 226, 10, 43, 209, 851, 1121, 1098, 572, 144, 84, 85], [277, 144, 84, 85], [4923, 144, 84, 85], [362, 277, 144, 84, 85], [294, 144, 84, 85], [872, 144, 84, 85], [294, 144, 84, 85], [632, 393, 471, 124, 129, 130, 132, 12, 16, 89, 1016, 1259, 96, 1085, 34, 226, 43, 1121, 572, 177, 144, 84, 85], [4923, 144, 84, 85], [124, 277, 149, 84, 85]]]]

#print ( max(x, key=lambda xmb: len(xmb[1])))

In [ ]:
class CNN_EHR(nn.Module):
    
    def __init__ (self, pred_cnt, embed_dim, ch_out, kernel_sizes, dropout ,eb_mode):
        super(CNN_EHR, self).__init__()
        
        P = pred_cnt ## number of predictors in our model
        D = embed_dim
        self.D = D
        C = 1
        Ci = 1
        Co = ch_out ## number of Channels out
        Ks = kernel_sizes ## a list for different kernel(filters dimension)

        self.embedBag = nn.EmbeddingBag(P, D,mode= eb_mode)
        self.convs1 = nn.ModuleList([nn.Conv1d(D, Co, K, padding=2) for K in Ks])
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(len(Ks)*Co, C)
        self.sigmoid = nn.Sigmoid()
   
    def EmbedPatient_MB(self, seq_mini_batch): # x is a ehr_seq_tensor
        
       
        lp= len(max(seq_mini_batch, key=lambda xmb: len(xmb[1]))[1])
        #print ('longest',lp)
        tb= torch.FloatTensor(len(seq_mini_batch),lp,self.D) 
        lbt1= torch.FloatTensor(len(seq_mini_batch),1)

        for pt in range(len(seq_mini_batch)):
              
            lbt ,pt_visits =seq_mini_batch[pt]
            lbt1[pt] = torch.FloatTensor([[float(lbt)]])
            ml=(len(max(pt_visits, key=len))) ## getting the max number of visits for pts within the minibatch
            txs= torch.LongTensor(len(pt_visits),ml)
            
            b=0
            for i in pt_visits:
                pd=(0, ml-len(i))
                txs[b] = F.pad(torch.from_numpy(np.asarray(i)).view(1,-1),pd,"constant", 0).data
                b=b+1
            
            if use_cuda:
                txs=txs.cuda()
                
            emb_bp= self.embedBag(Variable(txs)) ### embed will be num_of_visits*max_num_codes*embed_dim 
            #### the embed Bag dim will be num_of_visits*embed_dim
            
            zp= nn.ZeroPad2d((0,0,0,(lp-len(pt_visits))))
            xzp= zp(emb_bp)
            tb[pt]=xzp.data

        tb= tb.permute(1, 0, 2) ### as my final input need to be seq_len x batch_size x input_size
        emb_m=Variable(tb)
        label_tensor = Variable(lbt1)

        if use_cuda:
                label_tensor = label_tensor.cuda()
                emb_m = emb_m.cuda()
                
        return emb_m , label_tensor


    def forward(self, input):
        #x = self.EmbedPatient(input) # [seqlen*batchsize*embdim]

        x , lt = self.EmbedPatient_MB(input)
        x = x.permute(1,2,0) # [N, Co, W]
        
        x = [F.relu(conv(x)) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)
        
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        y = self.sigmoid(logit)
        
        return y , lt

In [ ]:

model = CNN_EHR(pred_cnt=20000, embed_dim=128, ch_out=64, kernel_sizes=[3], dropout=0.1, eb_mode='sum')

if use_cuda:
    model = model.cuda()


In [ ]:
def train (mini_batch, criterion, optimizer):  
    
    model.zero_grad()
    output , label_tensor = model(mini_batch)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.data[0]


In [ ]:
def variableFromEHRSeq(ehr_seq):
    # ehr_seq is a list of list
    result = []
    if use_cuda:
        for i in range(len(ehr_seq)):
            result.append( Variable(torch.LongTensor([int(v) for v in ehr_seq[i]])).cuda() )
    # if use_cuda:
    #     return result.cuda()
    else:
        for i in range(len(ehr_seq)):
            result.append( Variable(torch.LongTensor([int(v) for v in ehr_seq[i]])) )

    return result

In [ ]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)






In [ ]:
def run_model_train(dataset,batch_size,learning_rate = 0.001 ):
    
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    #optimizer = optim.Adadelta(model.parameters(), '''lr=learning_rate,''' weight_decay=0)
    optimizer = optim.Adam(model.parameters(), weight_decay=0)
    
    dataset.sort(key=lambda pt:len(pt[1])) 
   
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    # print('number of Batches',n_batches)
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Print iter number, loss, name and guess
            #if iter % print_every == 0:
             #   print('%d %d%% (%s) %.4f ' % ( iter, iter/ n_batches * 100, timeSince(start), loss))

            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses

    
    

In [ ]:
def calculate_auc(test_model, dataset, batch_size=200):

    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []
    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = model(batch)
            y_hat.append(output.cpu().data.numpy()[0][0])
            labelVec.append(label_t.cpu().data.numpy()[0][0])
    #print (labelVec, y_hat)
    auc = roc_auc_score(labelVec, y_hat)
    return auc

In [ ]:
epochs=20
batch_size=200
current_loss_l=[]
all_losses_l=[]

for ep in range(epochs):
    current_loss_la,all_losses_la = run_model_train(train_sl,batch_size)
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    print ("Epoch ", ep," : ","Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc)
    current_loss_l.append(current_loss_la)
    all_losses_l.append (all_losses_la)

In [ ]:
#all_losses_l

In [ ]:
# plotting and diagnose
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    
for all_losses_a in all_losses_l:
    showPlot(all_losses_a)

In [ ]:
##from torchviz import make_dot, make_dot_from_trace
output , label_tensor = model(train_sl[0:10])
make_dot (output)

In [ ]:
model